# Частоты и смещения

#### Частота в  pandas состоит из базовой частоты и кратности. Базовая частота обычно обозначается строкой, например 'D' означает раз в день, а 'h' – раз в час. Для каждой базовой частоты определен объект, называемый смещением даты (date offset). Так, частоту «раз в час» можно представить классом Hour:

In [1]:
from pandas.tseries.offsets import Hour, Minute
hour = Hour()
hour

<Hour>

#### Для определения кратности смещения нужно задать целое число:

In [2]:
four_hours = Hour(4)
four_hours

<4 * Hours>

#### В большинстве случаев не приходится создавать такие объекты явно, достаточно использовать их строковые обозначения вида 'h' или '4h'. Наличие целого числа перед базовой частотой создает кратную частоту:

In [3]:
import pandas as pd
pd.date_range('2025-01-01', '2025-01-03 23:59', freq='4h')

DatetimeIndex(['2025-01-01 00:00:00', '2025-01-01 04:00:00',
               '2025-01-01 08:00:00', '2025-01-01 12:00:00',
               '2025-01-01 16:00:00', '2025-01-01 20:00:00',
               '2025-01-02 00:00:00', '2025-01-02 04:00:00',
               '2025-01-02 08:00:00', '2025-01-02 12:00:00',
               '2025-01-02 16:00:00', '2025-01-02 20:00:00',
               '2025-01-03 00:00:00', '2025-01-03 04:00:00',
               '2025-01-03 08:00:00', '2025-01-03 12:00:00',
               '2025-01-03 16:00:00', '2025-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4h')

#### Операция сложения позволяет объединить несколько смещений:

In [4]:
 Hour(2) + Minute(30)

<150 * Minutes>

#### Можно также задать частоту в виде строки '2h30min', что приводит к тому же результату, что и выше:

In [5]:
pd.date_range('2025-01-01', periods=10, freq='2h30min')

DatetimeIndex(['2025-01-01 00:00:00', '2025-01-01 02:30:00',
               '2025-01-01 05:00:00', '2025-01-01 07:30:00',
               '2025-01-01 10:00:00', '2025-01-01 12:30:00',
               '2025-01-01 15:00:00', '2025-01-01 17:30:00',
               '2025-01-01 20:00:00', '2025-01-01 22:30:00'],
              dtype='datetime64[ns]', freq='150min')

Некоторые частоты описывают неравноотстоящие моменты времени. Например, значения частот 'ME' (конец календарного месяца) и  'BM' (последний рабочий день месяца) зависят от числа дней в месяце, а в последнем случае также от того, заканчивается месяц рабочим или выходным днем.

DateOffset похож на Timedelta, который представляет собой период времени, но следует определенным правилам календарной продолжительности. Например, день Timedelta всегда будет увеличивать время на 24 часа, в то время как день DateOffset будет увеличивать время до того же самого времени на следующий день, независимо от того, составляет ли день 23, 24 или 25 часов из-за перехода на летнее время. Однако все подклассы DateOffset, имеющие размер час или меньше (Hour, Minute, Second, Milli, Micro, Nano), ведут себя как Timedelta и соблюдают абсолютное время.

DateOffset действует аналогично dateutil.relativedelta, сдвигая время даты на соответствующую указанную календарную продолжительность. Для выполнения сдвига может использоваться арифметический оператор (+), как было показано выше

In [6]:
# В этот день происходит переход на летнее время
ts = pd.Timestamp("2016-10-30 00:00:00", tz="Europe/Helsinki")

In [7]:
# Соблюдает абсолютное время
ts + pd.Timedelta(days=1)

Timestamp('2016-10-30 23:00:00+0200', tz='Europe/Helsinki')

In [8]:
# Соблюдает календарное время
ts + pd.DateOffset(days=1)

Timestamp('2016-10-31 00:00:00+0200', tz='Europe/Helsinki')

### Методы offsets.rollforward и offsets.rollback

DateOffsets дополнительно имеет методы rollforward и rollback для перемещения даты вперед или назад. Например, для рабочих дней (Business...) даты, приходящиеся на выходные (субботу и воскресенье), будут сдвинуты вперед до понедельника, поскольку смещения этого типа работают по будням.

In [9]:
ts = pd.Timestamp('2018-01-06 00:00:00')

In [10]:
ts.day_name()

'Saturday'

In [11]:
# смещение для рабочих дней - с понедельника по пятницу.
offset = pd.offsets.BusinessHour(start='09:00')

In [12]:
# приводит дату к ближайшей дате смещения (понедельник)
offset.rollforward(ts)

Timestamp('2018-01-08 09:00:00')

In [13]:
# Сначала дата приводится к ближайшей смещенной дате, а затем добавляется час
ts + offset

Timestamp('2018-01-08 10:00:00')

#### По умолчанию эти операции сохраняют информацию о времени (часах, минутах и т. д.). Чтобы вернуть время к полуночи, используйте normalize() до или после применения операции (в зависимости от того, хотите ли вы включить информацию о времени в операцию).

In [14]:
ts = pd.Timestamp('2014-01-01 09:00')
day = pd.offsets.Day()
day + ts

Timestamp('2014-01-02 09:00:00')

In [15]:
(day + ts).normalize()

Timestamp('2014-01-02 00:00:00')

In [16]:
ts = pd.Timestamp('2014-01-01 22:00')
hour = pd.offsets.Hour()
hour + ts

Timestamp('2014-01-01 23:00:00')

In [17]:
(hour + ts).normalize()

Timestamp('2014-01-01 00:00:00')

In [18]:
(hour + pd.Timestamp('2014-01-01 23:30')).normalize()

Timestamp('2014-01-02 00:00:00')

### Функция offsets.CustomBusinessDay - создание собственных рабочих дней

Функция Pandas tseries.offsets.CustomBusinessDay() используется для создания собственных рабочих дней. Подкласс DateOffset, представляющий возможно n пользовательских рабочих дней, исключая праздники.

В примере мы указываем, что хотим добавить 3 рабочих дня (n=3), также указываем, какие дни недели считать рабочими (weekmask = 'Mon Tue Wed Thu').

In [20]:
# создаем временную метку Timestamp
ts = pd.Timestamp('2024-04-23 10:15:00')

# создаем сдвиг offset
cbd = pd.tseries.offsets.CustomBusinessDay(n = 3, weekmask = 'Mon Tue Wed Thu')

# вывод на печать Timestamp
print(ts)

# вывод на печать offset
print(cbd)

2024-04-23 10:15:00
<3 * CustomBusinessDays>


Теперь мы добавим смещение к заданному объекту timestamp, чтобы получить нужную дату с учетом входных данных. Как видно из вывода, мы успешно создали смещение и добавили его к заданной временной метке - прибавив к дате 23 апреля 2024 года 10:15:00 три рабочих дня мы получили 29 апреля 2024 года 10:15:00.

In [21]:
# добавляем сдвиг offset к объекту Timestamp и выводим получившийся объект Timestamp
new_timestamp = ts + cbd
new_timestamp

Timestamp('2024-04-29 10:15:00')

#### Даты, связанные с неделей месяца
Полезный класс частот – «неделя месяца», обозначается строкой, начинающейся с WOM. Он позволяет получить, например, третью пятницу каждого месяца:

In [19]:
monthly_dates = pd.date_range('2024-01-01', '2024-09-01', freq='WOM-3FRI')
monthly_dates

DatetimeIndex(['2024-01-19', '2024-02-16', '2024-03-15', '2024-04-19',
               '2024-05-17', '2024-06-21', '2024-07-19', '2024-08-16'],
              dtype='datetime64[ns]', freq='WOM-3FRI')

https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects